In [1]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from unet import create_unet_model
from metrics import psnr

2023-12-27 22:42:04.285865: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 22:42:04.436420: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-27 22:42:04.438145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-27 22:42:05.401305: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Load data

In [4]:
input_folder = "./images/"

image_files = [f for f in os.listdir(input_folder) if f.endswith(".png") or f.endswith(".jpg")]

images_Y = [cv2.imread(os.path.join(input_folder, img)) for img in image_files]

images_X = [cv2.resize(img, (48, 48), interpolation=cv2.INTER_AREA) for img in images_Y]

images_X = [img / 255.0 for img in images_X]
images_Y_ = [img / 255.0 for img in images_Y]

images_X = np.array(images_X)
images_Y = np.array(images_Y)

X_train, X_test, Y_train, Y_test = train_test_split(images_X, images_Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(Y_train.shape)
print(type(X_train))

(4000, 48, 48, 3)
(4000, 96, 96, 3)
<class 'numpy.ndarray'>


In [5]:
model_unet = create_unet_model()
model_unet.summary()

2023-12-27 22:42:37.611693: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-27 22:42:37.612458: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 48, 48, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 48, 48, 64)   36928       ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 24, 24, 64)   0           ['conv2d_1[0][0]']               
                                                                                              

In [6]:
model_unet.compile(optimizer='adam', loss='mean_squared_error', metrics=[psnr])

In [ ]:
model_unet.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
 34/100 [=========>....................] - ETA: 1:48 - loss: 32246.0742 - psnr: -42.1699